In [1]:
import os
import sys
import numpy as np
import pandas as pd
import collections
from mirnylib import h5dict, genome, plotting
from hiclib import mapping, fragmentHiC, highResBinnedData

hello from new mapping


In [2]:
genome_db = genome.Genome('/home/shared_data/Annotation/UCSC/Mouse_Genome/MM9/Mus_musculus/UCSC/mm9/Sequence/WholeGenomeFasta/genome.fa')

In [3]:
pwd

u'/home/xli/Data/Haihui/CD8-HP/HiC_2019/Data/Naive'

In [4]:
fname_list = ['19092FL-06-01_S1_L00%d_mapped_reads.hdf5' % i for i in range(1,3)]
#fname_list = ['WT_CD8_p%d_mapped_reads.hdf5' % i for i in range(1,7)]
mapped_reads_list = []
for fname in fname_list:
    mapped_reads_list.append(h5dict.h5dict(fname))

combine_mapped_reads = h5dict.h5dict('WT_CD8_1910_mapped_reads.hdf5')

combine_mapped_reads['chrms1'] = np.hstack(map(lambda k: k['chrms1'], mapped_reads_list))
combine_mapped_reads['chrms2'] = np.hstack(map(lambda k: k['chrms2'], mapped_reads_list))
combine_mapped_reads['cuts1'] = np.hstack(map(lambda k: k['cuts1'], mapped_reads_list))
combine_mapped_reads['cuts2'] = np.hstack(map(lambda k: k['cuts2'], mapped_reads_list))
combine_mapped_reads['misc'] = map(lambda k: k['misc'], mapped_reads_list)[0]
combine_mapped_reads['strands1'] = np.hstack(map(lambda k: k['strands1'], mapped_reads_list))
combine_mapped_reads['strands2'] = np.hstack(map(lambda k: k['strands1'], mapped_reads_list))


/home/xli/Downloads/mirnylab/mirnylib/h5dict.py:103: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  data = pickle.loads(self._h5file[self.self_key].value)
/home/xli/Downloads/mirnylab/mirnylib/h5dict.py:134: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  value = self._h5file[key].value


In [5]:
### from mapped reads to get fragment
fragments = fragmentHiC.HiCdataset(
    filename='WT_CD8_1910_fragment_dataset.hdf5',
    enzymeName='HindIII',
    genome=genome_db,
    mode='w')

fragments.parseInputData(
    dictLike='WT_CD8_1910_mapped_reads.hdf5', keepSingleSided=False, keepSameFragment=True, keepReadsMolecules=True)

----> New dataset opened, genome WholeGenomeFasta, filename = WT_CD8_1910_fragment_dataset.hdf5
     loading data from file WT_CD8_1910_mapped_reads.hdf5 (assuming h5dict)
raw read count file {0} does not exist
Count not find raw read count
filtering SS reads
Keeping same-fragment reads
keeping --> <-- reads even if they exceed maximumMoleculeLength
300_validPairs:  561051531


In [6]:
fragments.filterDuplicates(mode='ram')
fragments._sortData()

#fragments = fragmentHiC.HiCdataset('WT_CD8_1910_fragment_dataset.hdf5', genome_db, enzymeName='HindIII')

#fragments.filterDuplicates(mode='ram')

fragments.saveByChromosomeHeatmap('WT_CD8_1910_heatmap-res-100K.hdf5', resolution=100000)

#fragments.printMetadata(saveTo="DKO_CD8_2016_statistics.txt")

#print "# of Fragments after duplciate removal: " + str(fragments.fragmentSum(strands=1).sum())

----->Filtering duplicates in DS reads: 
Sorting using 2 chunks


/opt/miniconda2/lib/python2.7/site-packages/hiclib/fragmentHiC.py:600: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if outVariable[1] == "ToDefine":
/opt/miniconda2/lib/python2.7/site-packages/hiclib/fragmentHiC.py:600: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if outVariable[1] == "ToDefine":


----> By chromosome Heatmap saved to 'WT_CD8_1910_heatmap-res-100K.hdf5' at 100000 resolution


In [15]:
fragments.printMetadata()

In [16]:
### data for generating .hiC
#fragments = fragmentHiC.HiCdataset('DKO_CD8_fragment_dataset.hdf5', genome_db, enzymeName='HindIII')
outdir = '/home/xli/Data_Processing/Haihui/CD8-HP/HiC_2019/Data/201910/iterative_mapping_bam/Naive/DKO_CD8_1910/'
chrms1_array = np.empty_like(fragments.h5dict['chrms1'], dtype='|S5')
for index, chrmLabel in fragments.genome.idx2label.iteritems():
    chrms1_array[np.where(fragments.h5dict['chrms1'] == index)] = chrmLabel

chrms2_array = np.empty_like(fragments.h5dict['chrms2'], dtype='|S5')
for index, chrmLabel in fragments.genome.idx2label.iteritems():
    chrms2_array[np.where(fragments.h5dict['chrms2'] == index)] = chrmLabel

k = 10
index_list = np.linspace(0, fragments.N, k + 1, dtype=int)

for i in range(1,k+1):
    print i; sys.stdout.flush()
    start, end = index_list[i-1], index_list[i]
    df = pd.DataFrame({'str1': 1 - fragments.h5dict['strands1'][start : end],
                       'chr1': chrms1_array[start : end],
                       'pos1': fragments.h5dict['cuts1'][start : end],
                       'frag1': 0,
                       'str2': 1 - fragments.h5dict['strands2'][start : end],
                       'chr2': chrms2_array[start : end],
                       'pos2': fragments.h5dict['cuts2'][start : end],
                       'frag2': 1
                      },
                      columns=['str1','chr1','pos1','frag1','str2','chr2','pos2','frag2'])

    df.to_csv(outdir+'DKO_CD8_1910_Juicebox_input.txt', sep='\t', index=None, header=None, mode='a')

1
2
3
4
5
6
7
8
9
10


In [7]:
WT_CD8_BD = highResBinnedData.HiResHiC(genome_db, 100000, 'WT_CD8_1910_heatmap-res-100K.hdf5', mode='r+')
WT_CD8_BD.loadData('WT_CD8_1910_heatmap-res-100K.hdf5')
outdir = '/home/xli/Data/Haihui/CD8-HP/HiC_2019/Data/Naive_Merge_2016_1910/mapped_reads/matrix_res_100k/1910/WT_CD8/'
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'chrmLens': WT_CD8_BD.genome.chrmLens}).to_csv(outdir + 'mm9_genome_size.txt', index=None, header=None, sep='\t')
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'filename': map(lambda chrm: outdir + 'WT_CD8_' + chrm + '.matrix', WT_CD8_BD.genome.chrmLabels)}).to_csv(outdir + 'WT_CD8_intrachrom_matrix_list.txt', index=None, header=None, sep='\t')


In [12]:
WT_CD8_BD.removeDiagonal()
WT_CD8_BD.iterativeCorrection()

Pass = 0, Error = 2.364650
Pass = 1, Error = 0.980156
Pass = 2, Error = 0.862228
Pass = 3, Error = 0.660686
Pass = 4, Error = 0.498827
Pass = 5, Error = 0.399465
Pass = 6, Error = 0.336104
Pass = 7, Error = 0.290996
Pass = 8, Error = 0.257162
Pass = 9, Error = 0.229817
Pass = 10, Error = 0.207530
Pass = 11, Error = 0.189336
Pass = 12, Error = 0.173901
Pass = 13, Error = 0.160792
Pass = 14, Error = 0.149397
Pass = 15, Error = 0.139430
Pass = 16, Error = 0.130689
Pass = 17, Error = 0.123026
Pass = 18, Error = 0.116149
Pass = 19, Error = 0.109991
Pass = 20, Error = 0.104490
Pass = 21, Error = 0.099481
Pass = 22, Error = 0.094913
Pass = 23, Error = 0.090750
Pass = 24, Error = 0.086923
Pass = 25, Error = 0.083409
Pass = 26, Error = 0.080165
Pass = 27, Error = 0.077148
Pass = 28, Error = 0.074347
Pass = 29, Error = 0.071743
Pass = 30, Error = 0.069317
Pass = 31, Error = 0.067042
Pass = 32, Error = 0.064911
Pass = 33, Error = 0.062909
Pass = 34, Error = 0.061022
Pass = 35, Error = 0.059247
Pa

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
WT_CD8_BD.saveCooler('DKO_CD8_1910-res-10K.cool', 10000)

Exception AttributeError: "'h5dict' object has no attribute '_h5file'" in <bound method h5dict.__del__ of <mirnylib.h5dict.h5dict object at 0x7f44df4823d0>> ignored


AttributeError: 'HiResHiC' object has no attribute 'saveCooler'

In [8]:
for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    print key
    outputfile = 'WT_CD8_1910_' +  chrmLabel + '.matrix'
    
    
    np.savetxt(outdir + outputfile, WT_CD8_BD._h5dict[key], fmt='%.2e', delimiter='\t')

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 5)
(6, 6)
(7, 7)
(8, 8)
(9, 9)
(10, 10)
(11, 11)
(12, 12)
(13, 13)
(14, 14)
(15, 15)
(16, 16)
(17, 17)
(18, 18)
(19, 19)
(20, 20)
(21, 21)


## TKO

In [12]:
fragments = fragmentHiC.HiCdataset(
    filename='Tcf1_KO_na_CD8_fragment_dataset.hdf5',
    enzymeName='HindIII',
    genome=genome_db,
    mode='w')

fragments.parseInputData(
    dictLike='Tcf1_KO_na_CD8_mapped_reads.hdf5', keepSingleSided=False, keepSameFragment=True, keepReadsMolecules=True)


----> New dataset opened, genome iterative_mapping, filename = Tcf1_KO_na_CD8_fragment_dataset.hdf5
----->!!!File already exists! It will be deleted

     loading data from file Tcf1_KO_na_CD8_mapped_reads.hdf5 (assuming h5dict)
raw read count file {0} does not exist
Count not find raw read count
filtering SS reads
Keeping same-fragment reads
keeping --> <-- reads even if they exceed maximumMoleculeLength
300_validPairs:  196511151


In [29]:
fragments.fragmentSum(strands=1).sum()

131018607

In [13]:
fragments.filterDuplicates(mode='ram')
fragments._sortData()

fragments = fragmentHiC.HiCdataset('Tcf1_KO_na_CD8_fragment_dataset.hdf5', genome_db, enzymeName='HindIII')

fragments.saveByChromosomeHeatmap('Tcf1_KO_na_CD8_heatmap-res-10K.hdf5', resolution=10000)

fragments.printMetadata(saveTo="Tcf1_KO_na_CD8_statistics.txt")

----->Filtering duplicates in DS reads: 
Sorted using default sort 300000000 131018607
----> New dataset opened, genome iterative_mapping, filename = Tcf1_KO_na_CD8_fragment_dataset.hdf5
----->!!!File already exists! It will be opened in the append mode

----> By chromosome Heatmap saved to 'Tcf1_KO_na_CD8_heatmap-res-10K.hdf5' at 10000 resolution


In [14]:
TKO_CD8_BD = highResBinnedData.HiResHiC(genome_db, 10000, 'Tcf1_KO_na_CD8_heatmap-res-10K.hdf5', mode='r+')
TKO_CD8_BD.loadData('Tcf1_KO_na_CD8_heatmap-res-10K.hdf5')
outdir = '/home/lxiang/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/HiC/1905/iterative_mapping/Tcf1_KO_na_CD8'
pd.DataFrame({'chrmLabels': TKO_CD8_BD.genome.chrmLabels, 'chrmLens': TKO_CD8_BD.genome.chrmLens}).to_csv(outdir + '_mm9_genome_size.txt', index=None, header=None, sep='\t')
pd.DataFrame({'chrmLabels': TKO_CD8_BD.genome.chrmLabels, 'filename': map(lambda chrm: outdir + 'TKO_CD8_' + chrm + '.matrix', TKO_CD8_BD.genome.chrmLabels)}).to_csv(outdir + 'TKO_CD8_intrachrom_matrix_list.txt', index=None, header=None, sep='\t')


In [15]:
TKO_CD8_BD = highResBinnedData.HiResHiC(genome_db, 10000, 'Tcf1_KO_na_CD8_heatmap-res-10K-IC.hdf5')
TKO_CD8_BD.loadData('Tcf1_KO_na_CD8_heatmap-res-10K.hdf5')

In [ ]:
#TKO_CD8_BD.removeDiagonal()
TKO_CD8_BD.iterativeCorrection()

Pass = 0, Error = 2.149347
Pass = 1, Error = 0.966206
Pass = 2, Error = 0.823797
Pass = 3, Error = 0.622793
Pass = 4, Error = 0.471504
Pass = 5, Error = 0.379456
Pass = 6, Error = 0.321074
Pass = 7, Error = 0.279442
Pass = 8, Error = 0.247432
Pass = 9, Error = 0.221799
Pass = 10, Error = 0.200904
Pass = 11, Error = 0.183490
Pass = 12, Error = 0.168734
Pass = 13, Error = 0.156067
Pass = 14, Error = 0.145185
Pass = 15, Error = 0.135724
Pass = 16, Error = 0.127386
Pass = 17, Error = 0.119999
Pass = 18, Error = 0.113419
Pass = 19, Error = 0.107515
Pass = 20, Error = 0.102192
Pass = 21, Error = 0.097372
Pass = 22, Error = 0.092959
Pass = 23, Error = 0.088921
Pass = 24, Error = 0.085220
Pass = 25, Error = 0.081816
Pass = 26, Error = 0.078670
Pass = 27, Error = 0.075750
Pass = 28, Error = 0.073037
Pass = 29, Error = 0.070511
Pass = 30, Error = 0.068153
Pass = 31, Error = 0.065947
Pass = 32, Error = 0.063876
Pass = 33, Error = 0.061930
Pass = 34, Error = 0.060099
Pass = 35, Error = 0.058373
Pa

## Genome Size File

In [43]:
outdir = '/home/lxiang/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/HiC/1905/iterative_mapping/WT_na_CD8/'

In [44]:
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'chrmLens': WT_CD8_BD.genome.chrmLens}).to_csv(outdir + 'mm9_genome_size.txt', index=None, header=None, sep='\t')

In [45]:
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'filename': map(lambda chrm: outdir + 'WT_CD8_' + chrm + '.matrix', WT_CD8_BD.genome.chrmLabels)}).to_csv(outdir + 'WT_CD8_intrachrom_matrix_list.txt', index=None, header=None, sep='\t')

In [ ]:
fragments = fragmentHiC.HiCdataset('WT_CD8_fragment_dataset.hdf5', genome_db, enzymeName='HindIII')

fragments.saveByChromosomeHeatmap('WT_CD8_heatmap-res-10K.hdf5', resolution=10000)

In [ ]:
def from_mapping_to_fragment_hdf(BAM_PATH, Cond_NAME):
## Create a Class for storing
    fragments = fragmentHiC.HiCdataset( filename=Cond_NAME+'_fragment_dataset.hdf5',
        enzymeName='HindIII',genome=genome_db, mode='w')

## Read from mapped_reads to hdf5
    fragments.parseInputData(
        dictLike= Cond_NAME+'_mapped_reads.hdf5', keepSingleSided=False, keepSameFragment=True, keepReadsMolecules=True)
    
    

----> New dataset opened, genome iterative_mapping, filename = WT_CD8_fragment_dataset.hdf5


In [ ]:
WT_CD8_fragments = fragmentHiC.HiCdataset(
    filename='WT_CD8_fragment_dataset.hdf5',
    enzymeName='HindIII',
    genome=genome_db,
    mode='r')
#WT_CD8_fragments.fragmentSum(strands=1).sum()
WT_CD8_fragments.saveByChromosomeHeatmap('WT_CD8_heatmap-res-10k.hdf5', resolution=10000)
WT_CD8_BD = highResBinnedData.HiResHiC(genome_db, 10000, 'WT_CD8_read_count_heatmap-res-10K.hdf5', mode='a')
WT_CD8_BD.loadData('WT_CD8_heatmap-res-10k.hdf5')

for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    outputfile = 'WT_CD8_1905_read_count_' +  chrmLabel + '.matrix'
    np.savetxt(outdir + outputfile, WT_CD8_BD._h5dict[key], fmt='%i', delimiter='\t') 
    
# fmt='%.2e' for normalized matrix 
# fmt='%i' for raw read count

----> New dataset opened, genome iterative_mapping, filename = WT_CD8_fragment_dataset.hdf5
----> By chromosome Heatmap saved to 'WT_CD8_heatmap-res-10k.hdf5' at 10000 resolution


In [ ]:
TKO_CD8_fragments = fragmentHiC.HiCdataset(
    filename='Tcf1_KO_na_CD8_fragment_dataset.hdf5',
    enzymeName='HindIII',
    genome=genome_db,
    mode='r')

#WT_CD8_fragments.fragmentSum(strands=1).sum()
TKO_CD8_fragments.saveByChromosomeHeatmap('TKO_CD8_heatmap-res-10k.hdf5', resolution=10000)
TKO_CD8_BD = highResBinnedData.HiResHiC(genome_db, 10000, 'TKO_CD8_read_count_heatmap-res-10K.hdf5', mode='a')
TKO_CD8_BD.loadData('TKO_CD8_heatmap-res-10k.hdf5')

for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    outputfile = 'TKO_CD8_1905_read_count_' +  chrmLabel + '.matrix'
    np.savetxt(outdir + outputfile, TKO_CD8_BD._h5dict[key] , fmt='%i',  delimiter='\t')

In [122]:
total=0
for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    #print index, chrmLabel
    key = '(%d, %d)' % (index, index)
    total += WT_CD8_BD._h5dict[key].sum()
print total

162305580.0


In [147]:
pwd

u'/home/lxiang/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/HiC/1905/iterative_mapping'

In [87]:
test.sum()

12220.44415363108

In [47]:
for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    outputfile = 'WT_CD8_1905_' +  chrmLabel + '.matrix'
    np.savetxt(outdir + outputfile, WT_CD8_BD._h5dict[key], fmt='%.2e', delimiter='\t')

In [48]:
outdir = '/home/lxiang/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/HiC/1905/iterative_mapping/Tcf1_KO_na_CD8/'
for index, chrmLabel in TKO_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    outputfile = 'TKO_CD8_1905_' +  chrmLabel + '.matrix'
    np.savetxt(outdir+outputfile, TKO_CD8_BD._h5dict[key], fmt='%.2e', delimiter='\t')